In [2]:
import pandas as pd
import fidap
from config import api_key
import altair as alt

# instantiate api connection
fidap = fidap.fidap_client(api_key=api_key)

### Public Holidays in 2022  
  
Are the same holidays celebrated around the world? Do some countries have more public holidays than others? Are some weeks of the year more likely to have public holidays?  
  
Granted, our dataset does not have a lot of countries to start with, just 34. But we can start by looking at the number of holidays that these 34 countries have.    

In [6]:
ph_global_2022 = fidap.sql("""
SELECT COUNT(*) AS n_public_holidays, ph.countryRegionCode, c.country
FROM fidap-301014.azure_open_datasets.public_holidays AS ph
INNER JOIN gdelt-bq.extra.countryinfo2 AS c
ON ph.countryRegionCode = c.iso
WHERE EXTRACT(YEAR FROM ph.date) = 2022
AND ph.countryRegionCode IS NOT NULL
GROUP BY ph.countryRegionCode, c.country
ORDER BY n_public_holidays DESC;
""")

In [11]:
bar = alt.Chart(ph_global_2022).mark_bar().encode(
    x = 'n_public_holidays',
    y = alt.Y('country', sort = '-x')
)
rule = alt.Chart(ph_global_2022).mark_rule(color = 'red').encode(
    x = 'mean(n_public_holidays)'
)

(bar + rule).properties(height = 700)

alt.LayerChart(...)

Next, we can look at the weeks of 2022 when there will be a public holiday. 

In [12]:
n_holidays_by_week = fidap.sql("""
SELECT COUNT(*) AS n_holidays, EXTRACT(WEEK FROM ph.date) AS n_week
FROM fidap-301014.azure_open_datasets.public_holidays AS ph
WHERE EXTRACT(YEAR FROM ph.date) = 2022
GROUP BY n_week;
""")

In [13]:
alt.Chart(n_holidays_by_week).mark_line(interpolate = 'step-after').encode(
    x = 'n_week',
    y = 'n_holidays'
)

alt.Chart(...)

### Public Holidays in US  
  
Finally, we can also look at the number of public holidays in the US in 2021.   

In [16]:
us_holdays = fidap.sql("""
SELECT holidayName, SUBSTR(CAST(ph.date AS STRING),1,10) AS holidayDate   
FROM fidap-301014.azure_open_datasets.public_holidays AS ph
WHERE EXTRACT(YEAR FROM ph.date) = 2021
AND ph.countryRegionCode = 'US'
ORDER BY holidayDate
""")

In [17]:
us_holdays

,holidayName,holidayDate
0,New Year's Day,2021-01-01
1,Martin Luther King Jr. Day,2021-01-18
2,Washington's Birthday,2021-02-15
3,Memorial Day,2021-05-31
4,Independence Day,2021-07-04
5,Independence Day (Observed),2021-07-05
6,Labor Day,2021-09-06
7,Columbus Day,2021-10-11
8,Veterans Day,2021-11-11
9,Thanksgiving,2021-11-25
